In [12]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

## Multiplying two matrices

In [13]:
x = tf.random_normal([10,15])
y = tf.random_normal([15,10])

z = tf.matmul(x,y)

print("tf handle: ",z)

with tf.Session() as sess:
    res = sess.run(z)
    print("actual matrix product ", res)


tf handle:  Tensor("MatMul:0", shape=(10, 10), dtype=float32)
actual matrix product  [[ 0.51359665  2.872926    4.546303    3.3981328   0.75388277  0.30101556
   3.2338479   3.7337859   0.65348697 -0.64218354]
 [ 2.2632537  -2.7244818  -2.7225685  -2.1101034  -7.6224623  -4.579688
   5.278821    0.61067617  0.0966301  -0.30087176]
 [ 3.9118042   4.208037    9.509781    6.4187527   5.7446055   3.2369027
   9.91751     3.0986564  -1.9311206   8.5680685 ]
 [ 1.0557564   0.24406815  0.6063826   0.32787287  1.2544739  -4.111596
  -0.9338187   4.2576733   0.6308284  -0.8230331 ]
 [-1.4215841   1.2676978  -6.1712785   1.374303   -3.4528756  -5.5349593
  -0.570045    1.2239252   4.885189   -7.6845975 ]
 [-0.21696456  1.5884198   0.06801377  2.7661834   0.43136168 -0.42676637
   2.4129086   1.1250029   1.5321157   3.7543886 ]
 [ 1.7517694   1.2805418  -1.9844838   0.28716955 -4.3618317  -1.612256
   8.58833     0.05009779  0.2663226   1.2788968 ]
 [ 2.413901    1.4912431   2.7859695   0.3492698

## Learning a quadratic function 

We learn the function based on the input sample points using Stochastic Gradient Descent. 

First we define the model with placeholder input & outputs

In [14]:
# Placeholders are used to feed values from python to TensorFlow ops. We define
# two placeholders, one for input feature x, and one for output y.
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# Assuming we know that the desired function is quadratic in nature, i.e., a polynomial of 2nd degree, we
# allocate a vector of size 3 to hold the coefficients. The variable will be
# automatically initialized with random noise.
w = tf.get_variable("w", shape=[3, 1])

# We define yhat to be our estimate of y.
f = tf.stack([tf.square(x), x, tf.ones_like(x)], 1)
yhat = tf.squeeze(tf.matmul(f, w), 1)

Next, we add the necessary ingredients for performing stochastic descent (i.e., defining the optimization & the loss function)

In [21]:
# The loss is defined to be the l2 distance between our estimate of y and its
# true value. We also added a shrinkage term, to ensure the resulting weights
# would be small.
loss = tf.nn.l2_loss(yhat - y) + 0.1 * tf.nn.l2_loss(w)

# We use the Adam optimizer with learning rate set to 0.1 to minimize the loss.
train_op = tf.train.AdamOptimizer(0.1).minimize(loss)


In [27]:
# generate input data by sampling from some quadratic function 
# here we are trying to learn the function:
# f(x) = ax^2 + bx + c
def generate_quadratic(a,b,c):
    x_val = np.random.uniform(-10.0, 10.0, size=100)
    y_val = a * np.square(x_val) + b * x_val + c
    return x_val, y_val


In [28]:
a = 3
b = 2
c = 15
with tf.Session() as sess:
    # Since we are using variables we first need to initialize them.
    sess.run(tf.global_variables_initializer())
    for _ in range(1000): # Try replacing 1k with 10k. 
        x_val, y_val = generate_quadratic(a,b,c)
        _, loss_val = sess.run([train_op, loss], {x: x_val, y: y_val})
        #print(loss_val)
    print(sess.run([w]))

[array([[ 3.0162432],
       [ 2.0024037],
       [13.997231 ]], dtype=float32)]


## Shapes and reshaping

In [53]:
# returns shape (static if available or dynamic) of the tensor
def get_shape(tensor):
  static_shape = tensor.shape.as_list()
  dynamic_shape = tf.unstack(tf.shape(tensor))
  dims = [s[1] if s[0] is None else s[0]
          for s in zip(static_shape, dynamic_shape)]
  return dims

In [54]:
# reshapes the tensor by collapsing any number of dimensions
# this comes in very handy if you are dealing with input images that can be of multiple dimensions
# and are usually collapsed into 2 dimensions.
def reshape(tensor, dims_list):
  shape = get_shape(tensor)
  dims_prod = []
  for dims in dims_list:
    if isinstance(dims, int):
      dims_prod.append(shape[dims])
    elif all([isinstance(shape[d], int) for d in dims]):
      dims_prod.append(np.prod([shape[d] for d in dims]))
    else:
      dims_prod.append(tf.prod([shape[d] for d in dims]))
  tensor = tf.reshape(tensor, dims_prod)
  return tensor

## Sources:

[EffectiveTensorflow](https://github.com/vahidk/EffectiveTensorflow)